In [ ]:
def plot(df_sample, title):
    from bokeh.plotting import figure, show, output_file
    from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
    from bokeh.layouts import column
    from bokeh.models import Range1d, HoverTool
    from bokeh.models.formatters import DatetimeTickFormatter
    try:
        from bokeh.models import CustomJSTickFormatter
    except ImportError:  # Bokeh < 3.0
        from bokeh.models import FuncTickFormatter as CustomJSTickFormatter  # type: ignore

    # add a HoverTool to the default tools
    hover_tool = HoverTool(
        tooltips=[('x', '@x'), ('y', '@y'), ('value', '$y')])
    hover_tool_main = HoverTool(
        tooltips=[("x", "@{%F}"), ("y", "@y{$0,0.00}")], formatters={"@{%F}": "datetime"})

    # Create a ColumnDataSource object to hold the plot data
    source = ColumnDataSource(data=dict(
        x=df_sample.index,
        y=df_sample['Close'],
        bb_upper=df_sample['bb_upper'],
        bb_lower=df_sample['bb_lower'],
        signal=df_sample['signal'],
        false_signal=df_sample['false_signal'],
        average_adr=df_sample['average_adr'],
        macd=df_sample['macd'],
        macd_signal=df_sample['macd_signal'],
        macd_histogram=df_sample['macd_histogram'],
        rsi=df_sample['rsi'],
        bb_dist=df_sample['bb_distance'],
        vol=df_sample['volume']
    ))

    # Create views of the source using BooleanFilters
    # false_buy_filter = BooleanFilter([True if s==1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
    # false_sell_filter = BooleanFilter([True if s==-1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
    # false_buy_view = CDSView(source=source, filters=[false_buy_filter])
    # false_sell_view = CDSView(source=source, filters=[false_sell_filter])
    buy_filter = BooleanFilter([True if s == 1 and fs == 0 else False for s, fs in zip(
        source.data['signal'], source.data['false_signal'])])
    sell_filter = BooleanFilter([True if s == -1 and fs == 0 else False for s,
                                fs in zip(source.data['signal'], source.data['false_signal'])])
    buy_view = CDSView(source=source, filters=[buy_filter])
    sell_view = CDSView(source=source, filters=[sell_filter])

    # Create a Range1d object for the x-axis range shared by all three figures
    x_range = Range1d(start=df_sample.index[0], end=df_sample.index[-1])

    # Create a new figure object
    p = figure(title=title, x_axis_label='Date',
               y_axis_label='Price', width=1400, x_range=x_range)

    # add a vertical line at x=target
    # FIXME - this is not working
    # p.ray(x=target, y=0, length=100, angle=90,
    #       line_color="orange", line_width=20)

    # configure the x-axis to display datetime values
#     p.xaxis.formatter = CustomJSTickFormatter(
#         args=dict(axis=p.xaxis[0],
#                   formatter=DatetimeTickFormatter(days='%a, %d %b',
#                                                   months='%m/%Y'),
#                   source=source),
#         code='''
# this.labels = this.labels || formatter.doFormat(ticks
#                                                 .map(i => source.data.datetime[i])
#                                                 .filter(t => t !== undefined));
# return this.labels[index] || "";
#         ''')
    p.add_tools(hover_tool)

    # Plot the Close values as a line
    p.line(x='x', y='y', source=source, legend_label='Close')

    # Plot the bb_upper and bb_lower values as shaded areas
    p.varea(x='x', y1='bb_upper', y2='bb_lower', source=source,
            alpha=0.2, legend_label='Bollinger Bands')

    # Mark the long signals with green triangles and the short signals with red triangles
    p.triangle(x='x', y='y', source=source, view=buy_view,
               color='green', size=10, legend_label='Buy Signal')
    p.inverted_triangle(x='x', y='y', source=source, view=sell_view,
                        color='red', size=10, legend_label='Sell Signal')

    # p.triangle(x='x', y='y', source=source, view=false_buy_view, color='black', size=10, legend_label='False Buy Signal')
    # p.inverted_triangle(x='x', y='y', source=source, view=false_sell_view, color='black', size=10, legend_label='False Sell Signal')

    p_macd = figure(title='MACD', x_axis_label='Date',
                    y_axis_label='Value', width=1400, x_range=x_range)
    p_macd.add_tools(hover_tool)
    # Plot the average ADR as a line
    p_macd.line(x='x', y='average_adr', source=source,
                color='orange', legend_label='Average ADR')

    # Plot the MACD as two lines (macd and signal)
    p_macd.line(x='x', y='macd', source=source,
                color='blue', legend_label='MACD')
    p_macd.line(x='x', y='macd_signal', source=source,
                color='red', legend_label='Signal')

    # Plot the MACD histogram as a bar chart
    p_macd.vbar(x='x', top='macd_histogram', source=source, width=1.0,
                alpha=0.2, color='gray', legend_label='Histogram')

    # Plot the RSI as a line in a separate figure
    p_rsi = figure(title='RSI', x_axis_label='Date',
                   y_axis_label='RSI Value', width=1400, x_range=x_range)
    p_rsi.line(x='x', y='rsi', source=source, color='purple')
    p_rsi.add_tools(hover_tool)

    # Plot the bb distance as a line in a separate figure
    p_bb_dist = figure(title='BB Dist', x_axis_label='Date',
                       y_axis_label='Value', width=1400, x_range=x_range)
    p_bb_dist.line(x='x', y='bb_dist', source=source, color='blue')
    p_bb_dist.add_tools(hover_tool)

    # Plot the Volume as a Vol in a separate figure
    p_vol = figure(title='Volume', x_axis_label='Date',
                   y_axis_label='Value', width=1400, x_range=x_range)
    p_vol.line(x='x', y='vol', source=source, color='blue')
    p_vol.add_tools(hover_tool)

    show(column(p, p_vol, p_bb_dist, p_macd, p_rsi))
    # Save the plot as an HTML file
    # output_file(f'../charts/{input_path}-stats.html')


In [ ]:
import os
import warnings
import pandas as pd
warnings.filterwarnings('ignore', category=FutureWarning)
folder_path = '../tmp-data/period-trades/'
files = [f for f in os.listdir(folder_path) if f.startswith('5perc') and f.endswith('.csv')]
dfs = []
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file))
    dfs.append(df)

trades_df = pd.concat(dfs, ignore_index=True)

In [ ]:
# trades_df = pd.read_csv('../tmp-data/trade-stats-all-in-one.csv')

In [ ]:
trades_df.drop(columns=['Unnamed: 0'], inplace=True)
trades_df.columns

In [ ]:
trades_df ['EntryTime'] = pd.to_datetime(trades_df['EntryTime'])
trades_df ['ExitTime'] = pd.to_datetime(trades_df['ExitTime'])
trades_df.head(1)

In [ ]:
import numpy as np

# Define function to extract segment
def get_segment(df, entry_time, exit_time, before=1, after=1):
    mask = (df.index >= entry_time - pd.Timedelta(minutes=before)) & \
           (df.index <= exit_time + pd.Timedelta(minutes=after))
    return df.loc[mask]

In [ ]:
highest_df = trades_df.copy()
highest_df.sort_values(by='PnL', ascending=False, inplace=True)
high_entry_exit_times = np.array(list(zip(highest_df['EntryTime'].head(3), highest_df['ExitTime'].head(3))))
high_entry_exit_times

In [ ]:
lowest_df = trades_df.copy()
lowest_df.sort_values(by='PnL', ascending=True, inplace=True)
low_entry_exit_times = np.array(list(zip(lowest_df['EntryTime'].head(3), lowest_df['ExitTime'].head(3))))
low_entry_exit_times

In [ ]:
df_sample = pd.read_csv('../tmp-data/ss_range_bars.sell_above_adv.csv')
df_sample.reset_index(drop=True, inplace=True)
df_sample['timestamp'] = pd.to_datetime(df_sample['timestamp'])
df_sample = df_sample.sort_values(by='timestamp')
df_sample['ordinal'] = df_sample.index + 1
df_sample.set_index('timestamp', inplace=True)
df_sample.sort_index(inplace=True)

In [ ]:
# Create list of segments
segments = []
for low, high in zip(low_entry_exit_times, high_entry_exit_times):
    low_entry_time, low_exit_time = low
    high_entry_time, high_exit_time = high
    low_segment = get_segment(df_sample, low_entry_time, low_exit_time)
    segments.append([low_segment, 'low'])
    high_segment = get_segment(df_sample, high_entry_time, high_exit_time)
    segments.append([high_segment, 'high'])
    
print(len(segments))

In [ ]:
# seg, entry = segments[-1]
# plot(seg, entry)

In [ ]:
for seg, title in segments:
    plot(seg, title)